In [ ]:
import os

import math
import sys

import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as Func
from torch.nn import init
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

import torch.optim as optim

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from numpy import linalg as LA
import networkx as nx

from utils import *
from metrics import *
import pickle
import argparse
from torch import autograd
import torch.optim.lr_scheduler as lr_scheduler
from model import *

parser = argparse.ArgumentParser()


parser.add_argument('--input_size', type=int, default=2)
parser.add_argument('--output_size', type=int, default=5)
parser.add_argument('--n_stgcnn', type=int, default=1)
parser.add_argument('--kernel_size', type=int, default=3)
parser.add_argument('--num_channels', type=list, default=[14,12,10])

parser.add_argument('--obs_seq_len', type=int, default=20)
parser.add_argument('--pred_seq_len', type=int, default=10)
parser.add_argument('--dataset', default='Data1')
parser.add_argument('--dataset1', default='Data2')

parser.add_argument('--batch_size', type=int, default=128,
                    help='minibatch size')
parser.add_argument('--num_epochs', type=int, default=150,
                    help='number of epochs')
parser.add_argument('--clip_grad', type=float, default=None)
parser.add_argument('--lr', type=float, default=0.01,
                    help='learning rate')
parser.add_argument('--lr_sh_rate', type=int, default=50)
parser.add_argument('--use_lrschd', action="store_true", default=False)
parser.add_argument('--tag', default='20-10')
args = parser.parse_args(args=[])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('*' * 30)
print("Training initiating....")
print(args)


def graph_loss(V_pred, V_target):
    return bivariate_loss(V_pred, V_target)
size_list_path_ves = './datasets/ves.txt'

# Data prep
obs_seq_len = args.obs_seq_len
pred_seq_len = args.pred_seq_len
data_set = './datasets/' + args.dataset + '/'
data_set1 = './datasets/' + args.dataset1 + '/'
dset_train = TrajectoryDataset(
    data_set + 'train/',
    data_set1 + 'train/',
    size_list_path_ves,
    obs_len=obs_seq_len,
    pred_len=pred_seq_len,
    skip=1, norm_lap_matr=True)
torch.save(dset_train, "./datasets/dset_train20-10.pt")
dset_train = torch.load("./datasets/dset_train20-10.pt")

loader_train = DataLoader(
    dset_train,
    batch_size=1,
    shuffle=True,
    num_workers=0)

dset_val = TrajectoryDataset(
    data_set + 'val/',
    data_set1 + 'val/',
    size_list_path_ves,
    obs_len=obs_seq_len,
    pred_len=pred_seq_len,
    skip=1, norm_lap_matr=True)
torch.save(dset_val, "./datasets/dset_val20-10.pt")
dset_val = torch.load("./datasets/dset_val20-10.pt")
loader_val = DataLoader(
    dset_val,
    batch_size=1,
    shuffle=True,
    num_workers=1)

# Defining the model

model = social_stgcnn(n_stgcnn=args.n_stgcnn,
                      output_feat=args.output_size, seq_len=args.obs_seq_len,
                      kernel_size=args.kernel_size, pred_seq_len=args.pred_seq_len)

# Training settings

optimizer = optim.SGD(model.parameters(), lr=args.lr)

if args.use_lrschd:
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=args.lr_sh_rate, gamma=0.2)

checkpoint_dir = './checkpoint/' + args.tag + '/'

if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

with open(checkpoint_dir + 'args.pkl', 'wb') as fp:
    pickle.dump(args, fp)

print('Data and model loaded')
print('Checkpoint dir:', checkpoint_dir)

# Training
metrics = {'train_loss': [], 'val_loss': []}
constant_metrics = {'min_val_epoch': -1, 'min_val_loss': 9999999999999999}


def train(epoch):
    global metrics, loader_train
    model.train()
    loss_batch = 0
    batch_count = 0
    is_fst_loss = True
    loader_len = len(loader_train)
    turn_point = int(loader_len / args.batch_size) * args.batch_size + loader_len % args.batch_size - 1

    for cnt, batch in enumerate(loader_train):
        batch_count += 1

        # Get data
        batch = [tensor.to(device) for tensor in batch]
        obs_traj, pred_traj_gt, obs_traj_rel, pred_traj_gt_rel, non_linear_ped, \
        loss_mask, V_obs, A_obs, V_tr, A_tr, A_DPCA_obs, A_DPCA_tr, A_TPCA_obs, A_TPCA_tr, \
        A_vs_obs, A_vs_tr,A_sim_obs, A_sim_tr = batch

        optimizer.zero_grad()
        V_obs_tmp = V_obs.permute(0, 3, 1, 2)

        V_pred, _ = model(V_obs_tmp, A_obs.squeeze(), A_DPCA_obs.squeeze(), A_TPCA_obs.squeeze(), A_vs_obs.squeeze(), A_sim_obs.squeeze())

        V_pred = V_pred.permute(0, 2, 3, 1)

        V_tr = V_tr.squeeze()
        A_tr = A_tr.squeeze()
        V_pred = V_pred.squeeze()

        if batch_count % args.batch_size != 0 and cnt != turn_point:
            l = graph_loss(V_pred, V_tr)
            if is_fst_loss:
                loss = l
                is_fst_loss = False
            else:
                loss += l

        else:
            loss = loss / args.batch_size
            is_fst_loss = True
            loss.backward()

            if args.clip_grad is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip_grad)

            optimizer.step()
            # Metrics
            loss_batch += loss.item()
            print('TRAIN:', '\t Epoch:', epoch, '\t Loss:', loss_batch / batch_count)

    metrics['train_loss'].append(loss_batch / batch_count)


def vald(epoch):
    global metrics, loader_val, constant_metrics
    model.eval()
    loss_batch = 0
    batch_count = 0
    is_fst_loss = True
    loader_len = len(loader_val)
    turn_point = int(loader_len / args.batch_size) * args.batch_size + loader_len % args.batch_size - 1

    for cnt, batch in enumerate(loader_val):
        batch_count += 1

        # Get data
        batch = [tensor.to(device) for tensor in batch]
        obs_traj, pred_traj_gt, obs_traj_rel, pred_traj_gt_rel, non_linear_ped, \
        loss_mask, V_obs, A_obs, V_tr, A_tr, A_DPCA_obs, A_DPCA_tr, A_TPCA_obs, A_TPCA_tr,\
        A_vs_obs, A_vs_tr,A_sim_obs, A_sim_tr = batch

        V_obs_tmp = V_obs.permute(0, 3, 1, 2)

        V_pred, _ = model(V_obs_tmp, A_obs.squeeze(), A_DPCA_obs.squeeze(), A_TPCA_obs.squeeze(), A_vs_obs.squeeze(), A_sim_obs.squeeze())

        V_pred = V_pred.permute(0, 2, 3, 1)

        V_tr = V_tr.squeeze()
        A_tr = A_tr.squeeze()
        V_pred = V_pred.squeeze()

        if batch_count % args.batch_size != 0 and cnt != turn_point:
            l = graph_loss(V_pred, V_tr)
            if is_fst_loss:
                loss = l
                is_fst_loss = False
            else:
                loss += l

        else:
            loss = loss / args.batch_size
            is_fst_loss = True
            # Metrics
            loss_batch += loss.item()
            print('VALD:', '\t Epoch:', epoch, '\t Loss:', loss_batch / batch_count)

    metrics['val_loss'].append(loss_batch / batch_count)

    if metrics['val_loss'][-1] < constant_metrics['min_val_loss']:
        constant_metrics['min_val_loss'] = metrics['val_loss'][-1]
        constant_metrics['min_val_epoch'] = epoch
        torch.save(model.state_dict(), checkpoint_dir + 'val_best.pth')  # OK


print('Training started ...')
for epoch in range(args.num_epochs):
    train(epoch)
    vald(epoch)
    if args.use_lrschd:
        scheduler.step()

    print('*' * 30)
    print('Epoch:', args.tag, ":", epoch)
    for k, v in metrics.items():
        if len(v) > 0:
            print(k, v[-1])

    print(constant_metrics)
    print('*' * 30)

    with open(checkpoint_dir + 'metrics.pkl', 'wb') as fp:
        pickle.dump(metrics, fp)

    with open(checkpoint_dir + 'constant_metrics.pkl', 'wb') as fp:
        pickle.dump(constant_metrics, fp)

In [ ]:
import os
import math
import sys
import torch
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import pickle
import argparse
import glob
import torch.distributions.multivariate_normal as torchdist
from utils import * 
from metrics import * 
from model import social_stgcnn
import copy
dset_test = TrajectoryDataset(
        data_set+'test/',
        data_set1+'test/',
        size_list_path_ves,
        obs_len=obs_seq_len,
        pred_len=pred_seq_len,
        skip=1,norm_lap_matr=True)

loader_test = DataLoader(
        dset_test,
        batch_size=1,
        shuffle =False,
        num_workers=1)
torch.save(dset_test, "./datasets/dset_test20-10.pt")
def test(KSTEPS=20):
    global loader_test,model
    model.eval()
    ade_bigls = []
    fde_bigls = []
    raw_data_dict = {}
    step =0 
    for batch in loader_test: 
        step+=1
        #Get data
        batch = [tensor.to(device) for tensor in batch]
        obs_traj, pred_traj_gt, obs_traj_rel, pred_traj_gt_rel, non_linear_ped, \
        loss_mask, V_obs, A_obs, V_tr, A_tr, A_DPCA_obs, A_DPCA_tr, A_TPCA_obs, A_TPCA_tr,\
        A_vs_obs, A_vs_tr,A_sim_obs, A_sim_tr  = batch


        num_of_objs = obs_traj_rel.shape[1]

        #Forward
        #V_obs = batch,seq,node,feat
        #V_obs_tmp = batch,feat,seq,node
        V_obs_tmp =V_obs.permute(0,3,1,2)

        V_pred,_ = model(V_obs_tmp,A_obs.squeeze(),A_DPCA_obs.squeeze(),A_TPCA_obs.squeeze(),A_vs_obs.squeeze(),A_sim_obs.squeeze())
        V_pred = V_pred.permute(0,2,3,1)

        V_tr = V_tr.squeeze()
        A_tr = A_tr.squeeze()
        V_pred = V_pred.squeeze()
        num_of_objs = obs_traj_rel.shape[1]
        V_pred,V_tr =  V_pred[:,:num_of_objs,:],V_tr[:,:num_of_objs,:]

        sx = torch.exp(V_pred[:,:,2]) #sx
        sy = torch.exp(V_pred[:,:,3]) #sy
        corr = torch.tanh(V_pred[:,:,4]) #corr
        
        cov = torch.zeros(V_pred.shape[0],V_pred.shape[1],2,2)
        cov[:,:,0,0]= sx*sx
        cov[:,:,0,1]= corr*sx*sy
        cov[:,:,1,0]= corr*sx*sy
        cov[:,:,1,1]= sy*sy
        mean = V_pred[:,:,0:2]
        mvnormal = torchdist.MultivariateNormal(mean,cov)
        
        #Now sample 20 samples
        ade_ls = {}
        fde_ls = {}

        V_x = seq_to_nodes(obs_traj.data.cpu().numpy().copy())
        V_x_rel_to_abs = nodes_rel_to_nodes_abs(V_obs.data.cpu().numpy().squeeze().copy(),
                                                 V_x[0,:,:].copy())

        V_y = seq_to_nodes(pred_traj_gt.data.cpu().numpy().copy())
        V_y_rel_to_abs = nodes_rel_to_nodes_abs(V_tr.data.cpu().numpy().squeeze().copy(),
                                                 V_x[-1,:,:].copy())
        
        raw_data_dict[step] = {}
        raw_data_dict[step]['obs'] = copy.deepcopy(V_x_rel_to_abs)
        raw_data_dict[step]['trgt'] = copy.deepcopy(V_y_rel_to_abs)
        raw_data_dict[step]['pred'] = []

        for n in range(num_of_objs):
            ade_ls[n]=[]
            fde_ls[n]=[]

        for k in range(KSTEPS):

            V_pred = mvnormal.sample()

            V_pred_rel_to_abs = nodes_rel_to_nodes_abs(V_pred.data.cpu().numpy().squeeze().copy(),
                                                     V_x[-1,:,:].copy())
            raw_data_dict[step]['pred'].append(copy.deepcopy(V_pred_rel_to_abs))
            
            for n in range(num_of_objs):
                pred = [] 
                target = []
                obsrvs = [] 
                number_of = []
                pred.append(V_pred_rel_to_abs[:,n:n+1,:])
                target.append(V_y_rel_to_abs[:,n:n+1,:])
                obsrvs.append(V_x_rel_to_abs[:,n:n+1,:])
                number_of.append(1)

                ade_ls[n].append(ade(pred,target,number_of))
                fde_ls[n].append(fde(pred,target,number_of))
        
        for n in range(num_of_objs):
            ade_bigls.append(min(ade_ls[n]))
            fde_bigls.append(min(fde_ls[n]))

    ade_ = sum(ade_bigls)/len(ade_bigls)
    fde_ = sum(fde_bigls)/len(fde_bigls)
    return ade_,fde_,raw_data_dict


paths = ['./checkpoint/*social-stgcnn*']
KSTEPS=20

print("*"*50)
print('Number of samples:',KSTEPS)
print("*"*50)
ade_ls = [] 
fde_ls = [] 
path = paths[-1]
exps = './checkpoint/20-10/'
print('Model being tested are:',exps)

exp_path = exps
print("*"*50)
print("Evaluating model:",exp_path)

model_path = exp_path+'/val_best.pth'
args_path = exp_path+'/args.pkl'
with open(args_path,'rb') as f: 
    args = pickle.load(f)

stats= exp_path+'/constant_metrics.pkl'
with open(stats,'rb') as f: 
    cm = pickle.load(f)
print("Stats:",cm)



#Data prep     
obs_seq_len = args.obs_seq_len
pred_seq_len = args.pred_seq_len
data_set = './datasets/'+args.dataset+'/'


dset_test = torch.load("./datasets/dset_test20-10.pt")
loader_test = DataLoader(
        dset_test,
        batch_size=1,
        shuffle =False,
        num_workers=1)



#Defining the model 
model = social_stgcnn(n_stgcnn =args.n_stgcnn,
output_feat=args.output_size,seq_len=args.obs_seq_len,
kernel_size=args.kernel_size,pred_seq_len=args.pred_seq_len)
model.load_state_dict(torch.load(model_path, map_location='cpu'))


ade_ =999999
fde_ =999999
print("Testing ....")
ad,fd,raw_data_dic_= test()

In [ ]:
def get_ade(forecasted_trajectory_Lon,forecasted_trajectory_Lat,gt_trajectory_Lon,gt_trajectory_Lat):
    pred_len = forecasted_trajectory_Lon.shape[0]
    ade = float(
        sum(
            math.sqrt(
                (forecasted_trajectory_Lon[i] - gt_trajectory_Lon[i]) ** 2
                + (forecasted_trajectory_Lat[i] - gt_trajectory_Lat[i]) ** 2
            )
            for i in range(pred_len)
        )
        / pred_len
    )
    return ade
def get_fde(forecasted_trajectory_Lon,forecasted_trajectory_Lat,gt_trajectory_Lon,gt_trajectory_Lat):
    fde = math.sqrt( #单条轨迹中最后一个轨迹点坐标与真值的欧氏距离
        (forecasted_trajectory_Lon[-1] - gt_trajectory_Lon[-1]) ** 2  
        + (forecasted_trajectory_Lat[-1] - gt_trajectory_Lat[-1]) ** 2
    )
    return fde
ade = []
fde = []
for i in range(0,3):
    ade1 = []
    fde1 = []
    for a in range(1,len(raw_data_dic_)):
        tag = np.array(raw_data_dic_[a]['trgt'])
        pre = np.array(raw_data_dic_[a]['pred'])
        x2 = []
        x3 = []
        for j in range(0,20):
            x2.append(get_ade(pre[j,:,i,0],pre[j,:,i,1],tag[:,i,0],tag[:,i,1]))
            x3.append(get_fde(pre[j,:,i,0],pre[j,:,i,1],tag[:,i,0],tag[:,i,1]))
        ade1.append(x2)
        fde1.append(x3)
    ade.append(ade1)
    fde.append(fde1)
print('mean',np.array(ade).flatten().mean())
print('std',np.array(ade).flatten().std())
print('mean',np.array(fde).flatten().mean())
print('std',np.array(fde).flatten().std())